In [1]:
from load_results import load_result_dataset
import pandas as pd
import numpy as np
pn1 = 'full_fine_tuning_50epochs_edge_paper_final2'
pn2 = 'full_fine_tuning_50epochs_paper_final2'
pn3 = 'none' #'full_fine_tuning_50epochs_budget'

final_data = load_result_dataset(pn1, pn2, pn3)
final_data = [{**d, 'ft_strategy': 'FFT (50 epochs)'} for d in final_data]

../results/none/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/none/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/none/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50epochs_paper_final2/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/none/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50epochs_paper_final2/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/none/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50

In [9]:
from process_database import process_grouped_df, process_rankings, compute_odds_ratio_by_group

grouped_df = process_grouped_df(final_data,   )
grouped_df = process_rankings(grouped_df)
grouped_df = grouped_df[ ('TOTAL', )].reset_index()
# grouped_df.sort_values(by="sum", ascending=False)
# grouped_df.to_csv("./fft50.csv")
# grouped_df

Percentage of NaN values: 0.00%


In [18]:
import matplotlib.pyplot as plt
from process_database import process_grouped_df, process_rankings
import plotly.graph_objects as go
import plotly.colors as pc
import numpy as np
import pandas as pd

# Rebuild radar data with config names included for each dataset's rank-1 configuration
radar_data_with_names = []

xtick_label_map = {
    "hybrid": "Hybrid",
    "fusion":"Fusion",
    "self-supervised": "Uni-SS",
    "self-supervised (multimodal)": "Multi-SS",
    "supervised": "Sup.",
    "supervised (robust)": "Robust",
    "supervised (multistep)": "Multistep",
    "fully attention": "attention",
    "fully convolutional": "convolution",
    "CLASSIC_AT": "Classic AT",
    "TRADES_v2": "TRADES",
    'fgvc-aircraft-2013b':'fgvc-aircraft',
    'uc-merced-land-use-dataset':'land-use',
}


datas= [ 'stanford_cars',
        'oxford-iiit-pet',
        'caltech101' ,
        'flowers-102' ,
        'fgvc-aircraft-2013b',
        'uc-merced-land-use-dataset'
         ]

target = 'borda'
results = []

grouped_df = process_grouped_df(final_data,   )
grouped_df = process_rankings(grouped_df)

for dataset in sorted(datas):

        sub_dataset = grouped_df[(dataset,)]
        sub_dataset = sub_dataset.reset_index()
        sub_dataset["rank1"] = sub_dataset['borda'].rank(ascending=False, method='min')
        sub_dataset["rank2"] = sub_dataset['geom'].rank(ascending=False, method='min')
        sub_dataset = sub_dataset[ sub_dataset['rank1'] == 1 ]
        if sub_dataset.shape[0] > 1:
              print("hey")
              sub_dataset = sub_dataset[ sub_dataset['rank2'] == 1 ] 
        metric_dict = {metric: float(sub_dataset[metric]) for metric in ['L1_acc', 'L2_acc', 'Linf_acc', 'clean_acc', 'common_acc']}
        metric_dict['backbone_name'] = sub_dataset.get("backbone_name").iloc[0]
        metric_dict['loss_function'] = sub_dataset.get("loss_function").iloc[0]
        metric_dict['dataset'] = dataset
        results.append( metric_dict )

results = pd.DataFrame(results)

# Define axis labels
categories = ['L1_acc', 'L2_acc', 'Linf_acc', 'common_acc', 'clean_acc']
symbol_labels = [r'T(1)', r'T(2)', r'T(∞)', r'T(common)', r'T(∅)']

# Compute area as sum of metrics (proxy for polygon area)
results['area'] = results[categories].sum(axis=1)
results_sorted = results.sort_values(by='area', ascending=False).reset_index(drop=True)

# Use Plotly's qualitative color set
colors = pc.qualitative.Set2

fig = go.Figure()

for i, (_, row) in enumerate(results_sorted.iterrows()):
    # Map values using xtick_label_map, fallback to original if not found
    dataset_label = xtick_label_map.get(row['dataset'], row['dataset'])
    backbone_label = xtick_label_map.get(row['backbone_name'], row['backbone_name'])
    loss_label = xtick_label_map.get(row['loss_function'], row['loss_function'])

    # Make dataset bold
    lbl = f"<b>{dataset_label}</b><br>{backbone_label}<br>+{loss_label}"

    values = [row[metric] for metric in categories]
    values += [values[0]]  # Close shape

    color = colors[i % len(colors)]

    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=symbol_labels + [symbol_labels[0]],
        fill='toself',
        name=lbl,
        fillcolor=color.replace('rgb', 'rgba').replace(')', ', 0.3)'),  # 30% fill opacity
        line=dict(color=color, width=4.5)
    ))

width_px = 450
height_px = 330  # slightly reduced to account for text

fig.update_layout(
    width=width_px,
    height=height_px,
    polar=dict(
        bgcolor='white',
        radialaxis=dict(
            visible=True,
            showticklabels=True,
            ticks='',
            gridcolor='lightgray',
            linecolor='lightgray',
            range=[0, 1.0],  # optional: to keep scale consistent
        ),
        angularaxis=dict(
            rotation=90,
            direction='clockwise',
            tickfont=dict(size=15),
            gridcolor='lightgray',
            linecolor='lightgray'
        ),
    ),
    legend=dict(
        font=dict(size=10),
        bgcolor='rgba(255,255,255,0)',
        x=0.2,
        y=0.92,
        xanchor='right',
        yanchor='top'
    ),
    paper_bgcolor='white',
    showlegend=True,
    margin=dict(l=0, r=30, t=0, b=0),
)
fig.show()
fig.write_image("paper_figures/best_performing_per_dataset.png", format='png', width=width_px, height=height_px, scale=2)

Percentage of NaN values: 0.00%
hey
hey


In [19]:
from PIL import Image, ImageChops

def autocrop_image(input_path, output_path, bgcolor='white'):
    img = Image.open(input_path).convert("RGB")

    # Create background image with same size and background color
    bg = Image.new("RGB", img.size, bgcolor)

    # Find difference and crop box
    diff = ImageChops.difference(img, bg)
    bbox = diff.getbbox()

    if bbox:
        img_cropped = img.crop(bbox)
        img_cropped.save(output_path)
    else:
        img.save(output_path)  # Nothing to crop

# Example
autocrop_image("./paper_figures/best_performing_per_dataset.png", "./paper_figures/best_performing_per_dataset.png")
